In [ ]:
class LSTM4(nn.Module):
    def __init__(self, z_dim, a_dim):
        super().__init__()
        self.lstm = nn.LSTM(z_dim, z_dim, batch_first=True)
        self.fc_out = nn.Linear(z_dim, a_dim)

    def forward(self, zt):
        yt, _ = self.lstm(zt)
        at = self.fc_out(yt) # output shape: (N, L, output_size)
        return at
    
class LSTM5(nn.Module):  #Numero de neuronas ocultas diferente a z_dim
    def __init__(self, z_dim, h_dim, a_dim):
        super().__init__()
        self.lstm = nn.LSTM(z_dim, h_dim, batch_first=True)
        self.fc_out = nn.Linear(h_dim, a_dim)

    def forward(self, zt):
        yt, _ = self.lstm(zt)
        at = self.fc_out(yt) # output shape: (N, L, output_size)
        return at
    
class LSTM6(nn.Module):  # añadir tanh a la salida
    def __init__(self, z_dim, h_dim, a_dim):
        super().__init__()
        self.lstm = nn.LSTM(z_dim, h_dim, batch_first=True)
        self.fc_out = nn.Linear(h_dim, a_dim)

    def forward(self, zt):
        yt, _ = self.lstm(zt)
        at = F.tanh(self.fc_out(yt)) # output shape: (N, L, output_size)
        return at
    
class LSTM7(nn.Module):  # 2 capas de LSTM
    def __init__(self, z_dim, h_dim, a_dim):
        super().__init__()
        self.lstm = nn.LSTM(z_dim, h_dim, num_layers=2, batch_first=True)
        self.fc_out = nn.Linear(h_dim, a_dim)

    def forward(self, zt):
        yt, _ = self.lstm(zt)
        at = F.tanh(self.fc_out(yt)) # output shape: (N, L, output_size)
        return at
    
class LSTM8wtanh(nn.Module):  
    def __init__(self, z_dim, h_dim, num_layers, a_dim):
        super().__init__()
        self.lstm = nn.LSTM(z_dim, h_dim, num_layers=num_layers, batch_first=True)
        self.fc_out = nn.Linear(h_dim, a_dim)

    def forward(self, zt):
        yt, _ = self.lstm(zt)
        at = F.tanh(self.fc_out(yt)) # output shape: (N, L, output_size)
        return at

In [ ]:
# entendiendo la diferencia entre output y h_n en LSTM

import torch
import torch.nn as nn

torch.manual_seed(0)
mmm = nn.LSTM( input_size = 1, hidden_size = 10, num_layers  = 1 )
output, (hn, cn) = mmm(torch.rand( 50, 1, 1))

print(output.shape, hn.shape)

print(output[-1,:,:])
print()
print(hn)

# Checa aca el dibujoo
# https://stackoverflow.com/questions/48302810/whats-the-difference-between-hidden-and-output-in-pytorch-lstm

In [ ]:
## Version 0.1 de la sincronizacion 
from monitorlib import load_pcdata
import pandas as pd

def sync_positions(data,sample,i):
     # Searching for the cursor position that it is nearest to screenshot timestamp ti 
     # (before or after, it does not matter)
     ti = data[sample]['screenshots']['timestamp'][i]
     abs_diff = abs(data[sample]['moves']['timestamp'] - ti)
     closest_index = abs_diff.idxmin()
     px, py = data[sample]['moves']['px'][closest_index], data[sample]['moves']['py'][closest_index]
     return px, py

def sync_MKevents(sample_df, data, sample):
     
     def set_value(sample_df, abs_diff, event):
          closest_index = abs_diff.idxmin()
          if sample_df['mouse_keyboard'][closest_index] != None:
               print('colision')
               # what to do?
               sample_df.loc[closest_index, 'mouse_keyboard'] = event
          else:
               sample_df.loc[closest_index, 'mouse_keyboard'] = event
          

     # Initial values
     sample_df['mouse_keyboard'] = [None for i in range(len(sample_df))]
     timestamps_df = data[sample]['screenshots']['timestamp']

     # Clicks
     for i in range(len(data[sample]['clicks'])):
          ti = data[sample]['clicks']['timestamp'][i]
          button = data[sample]['clicks']['button'][i]
          set_value(sample_df, abs(timestamps_df - ti), button)

     # Scrolls
     for i in range(len(data[sample]['scrolls'])):
          ti = data[sample]['scrolls']['timestamp'][i]
          dy = data[sample]['scrolls']['dy'][i] 
          if dy == 1:
               scroll = 'scroll_down' 
          elif dy == -1:
               scroll = 'scroll_up'
          else:
               scroll = 'what?_scroll'
          set_value(sample_df, abs(timestamps_df - ti), scroll)
   
     # Keyboard events
     for i in range(len(data[sample]['keys'])):
          ti = data[sample]['keys']['timestamp'][i]
          key = data[sample]['keys']['key'][i].strip("'")
          set_value(sample_df, abs(timestamps_df - ti), key)

def sync(data):
     samples = []
     for sample in data.keys():
          sample_df = pd.DataFrame()
          sample_df['img_path'] = [img_path for img_path in data[sample]['screenshots']['img_path']]
          
          # Sync cursor positions to screenshots
          positions = [sync_positions(data,sample,i) for i in range(len(data[sample]['screenshots']))]
          sample_df['px'] = [px for px, _ in positions]
          sample_df['py'] = [py for _, py in positions]

          # Sync mouse and keyboard events to screenshots
          sync_MKevents(sample_df, data, sample)

          samples.append(sample_df)
     return samples

data = load_pcdata('data') 

# Syncronization
samples = sync(data)

# Replace None values with a string
for sample in samples:
    sample.fillna("no_action",inplace=True)

# tokenization
tokens = set()
for sample in samples:
    tokens.update(sample['mouse_keyboard'].unique().tolist())
tokens = list(tokens)
tokens.sort()
print(len(tokens))
print(tokens)

# Add timestamp column
for sample in samples:
    sample['time'] = sample['img_path'].map(lambda x: float(x.split('/')[2].split('_')[-1].split('.jpg')[0]))

samples[0]

In [ ]:
### Version 0.0 de la sincronizacion 
from monitorlib import load_pcdata
import pandas as pd

# Tengo mal este codigo en la sincronizacion: porque hay mas clicks y teclas apretadas que as que deberia haber. 
# Debo buscar en lcicks y pressed_keys y asociarlas a los tiempos de las iamgenes.  No barrer sobre las imagenes. 
# lo mismo para los scrolls, quizas para las poscines dle curso no es necesairos porque son mas abundantes que las iamgenes

data = load_pcdata('data')

SyncTHclicks = 0.2
SyncTHscrolls = 0.3
SyncTHkeys = 0.3 

def sync(data,sample,i):
     # Cursor positions
     ti = data[sample]['screenshots']['timestamp'][i]
     abs_diff = abs(data[sample]['moves']['timestamp'] - ti)
     closest_index = abs_diff.idxmin()
     mx, my = data[sample]['moves']['px'][closest_index], data[sample]['moves']['py'][closest_index]

     # Clicks
     abs_diff = abs(data[sample]['clicks']['timestamp'] - ti)
     if abs_diff.min() < SyncTHclicks: 
          closest_index = abs_diff.idxmin()
          cx, cy = data[sample]['clicks']['px'][closest_index], data[sample]['clicks']['py'][closest_index]  
          button = data[sample]['clicks']['button'][closest_index]
     else:
          cx, cy, button = None, None, 'no action'

     # Scrolls
     abs_diff = abs(data[sample]['scrolls']['timestamp'] - ti)
     if abs_diff.min() < SyncTHscrolls: 
          closest_index = abs_diff.idxmin()
          dx, dy = data[sample]['scrolls']['dx'][closest_index], data[sample]['scrolls']['dy'][closest_index] 
          if dy == 1:
               scroll = 'scroll_down' 
          elif dy == -1:
               scroll = 'scroll_up'
          else:
               scroll = 'what?_scroll'
     else:
          scroll = 'no action'
          

     # Keyboard events
     abs_diff = abs(data[sample]['keys']['timestamp'] - ti)
     if abs_diff.min() < SyncTHkeys: 
          closest_index = abs_diff.idxmin()
          key = data[sample]['keys']['key'][closest_index].strip("'")
     else:
          key = 'no action'

     return mx, my, cx, cy, button, scroll, key

img_paths, peripherals = [], []
for sample in data.keys():
     img_paths.extend([img_path for img_path in data[sample]['screenshots']['img_path']])
     peripherals.extend(sync(data,sample,i) for i in range(len(data[sample]['screenshots'])))

pc_df = pd.DataFrame(img_paths, columns=['img_path'])
knames = ['mouse_px', 'mouse_py', 'click_px', 'click_py', 'click_button', 'scroll', 'pressed_key']
for i,kname in enumerate(knames):
     pc_df[kname] = [x[i] for x in peripherals]

pc_df

In [ ]:
# retardos puestos y debugging de las coordenadas manualmente
# para samples[0]
actions = [('Button.left', (28, 131), 0.01), 
           ('Button.left', (1754, 45),0.01),
           ('z', (1754, 45), 0.01),
           ('o', (1754, 46), 0.01),
           ('o', (1754, 46), 0.01), 
           ('Key.enter', (1754, 46), 6),
           ('Button.left', (1040, 600), 2),  #(_,567)
           ('Button.left', (1075, 275), 1),
           ('Button.left', (1017, 564), 1),
           ('Button.left', (1080, 294), 1),
           ('Button.left', (1100, 765), 5), #(1056, 725)
           ('Button.left', (946, 755), 1),
           ('Button.left', (943, 769), 1),
           ('Button.left', (1025, 790), 1),  #(998, 775)
           ('Button.left', (929, 828), 1),
           ('Button.left', (427, 1057), 1),
           ('Button.left', (665, 746), 0.01),
]

# Tool for debugging actions
from PIL import Image, ImageDraw

img = Image.open('notable1.png')
draw = ImageDraw.Draw(img)
points = []
for i, (_,point, _) in enumerate(actions, start=1):
    draw.text(point, str(i), fill='red')
    points.append(point)
draw.polygon(points, outline='red')
#img.show()

time.sleep(2)
for action in actions:
    pcontroller(action[0], action[1])
    time.sleep(action[2])

In [1]:
import torch

# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

Using device: cuda

NVIDIA GeForce RTX 4090
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


In [ ]:
### Probando Depth Anything

from PIL import Image
import numpy as np

x = np.asarray(Image.open('../../Downloads/rgb.jpg')).copy()
d = np.asarray(Image.open('../../Downloads/depth.png'))

mask = 255*np.ones((1080,1920,1))
mask[d > 50] = 0
n = np.concatenate([x,mask],axis=2)


img = Image.fromarray(n.astype('uint8'))
img.show()
img.save('../../Downloads/crop.png')

In [ ]:
### dEnfocando la mirada en lo importante e ignorando el resto

### pensandolo mejor conviene meter dos imagenes una zoom out y otra zoom in cuya posicion este controlada por el agente. 

from PIL import Image
import numpy as np

def focuscrop(x1, y1, dw, factor):
    def reduce(img, box, factor):
        im = img.crop(box)
        return im.resize((im.size[0]//factor,im.size[1]//factor))
    
    im = img.crop((x1,y1,x1+dw,y1+dw))
    im1 = reduce(img, (0,0,x1,h), factor)
    im2 = reduce(img, (x1+dw,0,w,h), factor)
    im3 = reduce(img, (x1,0,x1+dw,y1), factor)
    im4 = reduce(img, (x1,y1+dw,x1+dw,h), factor)
    return [im, im1, im2, im3, im4]

img = Image.open(data['sample1']['screenshots']['img_path'][4])
w, h = img.size
x = np.asarray(img).copy()

imgs = focuscrop(x1=1920//2, y1 = 1080//2, dw = 300, factor = 5)
[imgs[i].show() for i in range(len(imgs))]

npixels_1 = [imgs[i].size[0]*imgs[i].size[1]*3 for i in range(len(imgs))]
npixels_1 = np.asarray(npixels_1).sum()
npixels_o = img.size[0]*img.size[1]*3
print(npixels_o)
print(npixels_1, round(npixels_1/npixels_o,4)*100, '%')

In [ ]:
## Usan la api de open ai para interpretar el screenhoot

import base64
import requests

api_key =

# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

image_path = data['sample1']['screenshots']['img_path'][6]
base64_image = encode_image(image_path)
headers = {
  "Content-Type": "application/json",
  "Authorization": f"Bearer {api_key}"
}

payload = { 
  "model": "gpt-4-vision-preview",
  "messages": [
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "What’s in this image?"
        },
        {
          "type": "image_url",
          "image_url": {
            "url": f"data:image/jpeg;base64,{base64_image}"
          }
        }
      ]
    }
  ],
  "max_tokens": 300
}

response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

print(response.json()['choices'][0]['message']['content'])